In [1]:
from pyspark.sql import SparkSession

In [2]:
df = spark.read.csv('/FileStore/tables/cruise_ship_info.csv',inferSchema=True,header=True)

In [3]:
df.printSchema()

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)

In [4]:
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
 Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55|
Celebration| Carnival| 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7|
 Conquest| Carnival| 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1|
 Destiny| Carnival| 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0|
 Ecstasy| Carnival| 22| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Elation| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Fantasy| Carnival| 23| 70.367| 20.56| 8.55| 10.22| 34.23| 9.2|
Fascination| Carnival| 19| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Freedom| Carnival| 6|110.23899999999999| 37.0| 9.51| 14.87| 29.79|11.5|
 Glory| Carnival| 10| 110.0| 29.74| 9.51| 14.87| 36.99|11.6|
 Holiday| Carnival| 28| 46.052| 14.52| 7.27| 7.26| 31.72| 6.6|
Imagination| Carnival| 18| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
Inspiration| Carnival| 17| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Legend| Carnival| 11| 86.0| 21.24| 9.63| 10.62| 40.49| 9.3|
 Liberty*| Carnival| 8| 110.0| 29.74| 9.51| 14.87| 36.99|11.6|
 Miracle| Carnival| 9| 88.5| 21.24| 9.63| 10.62| 41.67|10.3|
 Paradise| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Pride| Carnival| 12| 88.5| 21.24| 9.63| 11.62| 41.67| 9.3|
 Sensation| Carnival| 20| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 20 rows

In [5]:
df.groupBy('Cruise_line').count().show()

+-----------------+-----+
 Cruise_line|count|
+-----------------+-----+
 Costa| 11|
 P&O| 6|
 Cunard| 3|
Regent_Seven_Seas| 5|
 MSC| 8|
 Carnival| 22|
 Crystal| 2|
 Orient| 1|
 Princess| 17|
 Silversea| 4|
 Seabourn| 3|
 Holland_American| 14|
 Windstar| 3|
 Disney| 2|
 Norwegian| 13|
 Oceania| 3|
 Azamara| 2|
 Celebrity| 10|
 Star| 6|
 Royal_Caribbean| 23|
+-----------------+-----+

In [6]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Cruise_line", outputCol="cruise_cat")
indexed = indexer.fit(df).transform(df)
indexed.head(5)

Out[ 6 ]: [Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0),
 Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, cruise_cat=1.0),
 Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0, cruise_cat=1.0)]

In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
assembler = VectorAssembler(
  inputCols=['Age',
             'Tonnage',
             'passengers',
             'length',
             'cabins',
             'passenger_density',
             'cruise_cat'],
    outputCol="features")

In [9]:
output = assembler.transform(indexed)

In [10]:
output.select("features", "crew").show()

+--------------------+----+
 features|crew|
+--------------------+----+
[6.0,30.276999999...|3.55|
[6.0,30.276999999...|3.55|
[26.0,47.262,14.8...| 6.7|
[11.0,110.0,29.74...|19.1|
[17.0,101.353,26....|10.0|
[22.0,70.367,20.5...| 9.2|
[15.0,70.367,20.5...| 9.2|
[23.0,70.367,20.5...| 9.2|
[19.0,70.367,20.5...| 9.2|
[6.0,110.23899999...|11.5|
[10.0,110.0,29.74...|11.6|
[28.0,46.052,14.5...| 6.6|
[18.0,70.367,20.5...| 9.2|
[17.0,70.367,20.5...| 9.2|
[11.0,86.0,21.24,...| 9.3|
[8.0,110.0,29.74,...|11.6|
[9.0,88.5,21.24,9...|10.3|
[15.0,70.367,20.5...| 9.2|
[12.0,88.5,21.24,...| 9.3|
[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows

In [11]:
final_data = output.select("features", "crew")

In [12]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [13]:
from pyspark.ml.regression import LinearRegression
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='crew')

In [14]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data)

In [15]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [-0.0132852417066,0.0148670767471,-0.152640290194,0.383137521658,0.800697690907,-0.0102861436619,0.0284204538096] Intercept: -0.2575822068369844

In [16]:
test_results = lrModel.evaluate(test_data)

In [17]:
predictions = lrModel.transform(test_data)

In [18]:
predictions.show(10)

+--------------------+-----+------------------+
 features| crew| prediction|
+--------------------+-----+------------------+
[5.0,133.5,39.59,...|13.13|12.764925965850448|
[6.0,112.0,38.0,9...| 10.9|11.020560721136887|
[6.0,113.0,37.82,...| 12.0|11.429822053605527|
[7.0,116.0,31.0,9...| 12.0|12.424587298050403|
[8.0,77.499,19.5,...| 9.0| 8.546363011783969|
[9.0,59.058,17.0,...| 7.4|7.4768577118507356|
[9.0,88.5,21.24,9...| 10.3| 9.48932780462084|
[9.0,90.09,25.01,...| 8.69| 9.219600706372187|
[9.0,113.0,26.74,...|12.38|11.192343365302667|
[9.0,116.0,26.0,9...| 11.0|11.029364965609675|
+--------------------+-----+------------------+
only showing top 10 rows

In [19]:
print("R2: {}".format(test_results.r2))

R2: 0.836758844132595